# Book 3 of 4: Working with Units



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Book-3-of-4:-Working-with-Units" data-toc-modified-id="Book-3-of-4:-Working-with-Units-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Book 3 of 4: Working with Units</a></span><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Learning Objectives</a></span></li><li><span><a href="#Previously-On:" data-toc-modified-id="Previously-On:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Previously On:</a></span></li><li><span><a href="#Checking-Units-Manually" data-toc-modified-id="Checking-Units-Manually-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Checking Units Manually</a></span></li><li><span><a href="#Using-cf_units" data-toc-modified-id="Using-cf_units-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Using <code>cf_units</code></a></span><ul class="toc-item"><li><span><a href="#-Task-1---Perform-unit-conversion-on-a-temperature-value-" data-toc-modified-id="-Task-1---Perform-unit-conversion-on-a-temperature-value--1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span> Task 1 - Perform unit conversion on a temperature value </a></span></li></ul></li><li><span><a href="#Using-apply_ufunc" data-toc-modified-id="Using-apply_ufunc-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Using <code>apply_ufunc</code></a></span><ul class="toc-item"><li><span><a href="#What-is-apply_ufunc?" data-toc-modified-id="What-is-apply_ufunc?-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>What is apply_ufunc?</a></span></li><li><span><a href="#-Task-2----Write-a-function-for-unit-conversion" data-toc-modified-id="-Task-2----Write-a-function-for-unit-conversion-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span> Task 2 -  Write a function for unit conversion</a></span></li><li><span><a href="#-Task-3----Call-up-your-unit-conversion-function" data-toc-modified-id="-Task-3----Call-up-your-unit-conversion-function-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span> Task 3 -  Call up your unit conversion function</a></span></li></ul></li><li><span><a href="#Going-Futher:" data-toc-modified-id="Going-Futher:-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Going Futher:</a></span></li></ul></li></ul></div>

## Learning Objectives

- Use Xarray's metadata and the `cfunits` package to convert units of dataset variables.

----------------

## Previously:

- We imported the Xarray module and loaded our data.
- We wrote a function to subselect by depth.

Before continuing, make sure the correct Python environment is running in your notebook.  See the beginning of Notebook 01 for more detailed information.

In [ ]:
import xarray as xr

path = '../../../data/'
file = path + 'thetao_Omon_historical_GISS-E2-1-G_r1i1p1f1_gn_185001-185512.nc'
ds = xr.open_dataset(file)

def limit_depth_of_variables(level_bounds, temperature, depth_limit): 
    level_bounds_limited = level_bounds.where(level_bounds < depth_limit, depth_limit)
    delta_level = abs(level_bounds_limited[:, 1] - level_bounds_limited[:, 0])
    
    delta_level_limited = delta_level.where(delta_level != 0, drop = True)
    temperature_limited = temperature.where(delta_level != 0, drop = True)
    
    return delta_level_limited, temperature_limited

delta_level_limited, temperature_limited = limit_depth_of_variables(ds['lev_bnds'], ds['thetao'], 50)

----------------

## Checking Units Manually

In an Xarray Dataset, units are stored as an *attribute* of a dataset variable.  You can access the units as follows:

In [ ]:
level_units = ds['thetao'].attrs['units']
level_units

Say we wanted to convert all temperature values from degrees Celsius to Kelvins. 

First we apply the conversion, then we add our new units attribute.

In [ ]:
heat_in_Kelvins = ds['thetao'] + 273.15
heat_in_Kelvins.attrs['units'] = 'degK'
heat_in_Kelvins

## Using `cf_units`

In many routines, checking and converting betweeen units is complicated. But in Python, we have a great tool `cf_units` that can do this quickly and easily as long as the data is CF-compliant.

**What is cf_units?** 

Cf_units is a package that stores, combines, and compares physical units, allowing the user to perform unit conversion. You can read more about this package [here](https://scitools.org.uk/cf-units/docs/latest/unit.html).

Let's import the `cf_units` library.

The first functionality we will use is `cf_units.Unit()`, where you can pass in a unit of measure either as a string or by pointing to an attribute containing units. Cf_units checks if this unit is supported and converts it to the class `Units`.

First, we will look at the units for a single point along the levels dimension.

In [ ]:
import cf_units as cf

level_point = ds['lev'][0]
level_point

In [ ]:
orig_units = cf.Unit(level_point.attrs['units'])
orig_units

Then we will use `cf_units.Unit.convert` to convert from our original units to our target units.

In [ ]:
target_units = cf.Unit('km')
orig_units.convert(level_point, target_units)

<h3 style="color:red"> Task 1 - Perform unit conversion on a temperature value </h3>

Use cf_units to convert the units of `thetao_point` into Kelvin (`degK`):

In [ ]:
thetao_point = ds['thetao'].isel(time=0, lev = 0, lat = 30, lon=30)

In [ ]:
# Your code here

In [ ]:
# %load solutions/solution_3_1.py

Cf_units is nice because you can tell the code to convert your data to a specific unit without needing to explicitly know your current units. This allows more consistency in your workflows, so you can use the same code for temperature in C or in K.

--------------

## Using `apply_ufunc`

You will notice that the `cf_units.Unit.convert` function caused us to lose the information contained in our Xarray DataArray. We will fix this by using `xarray.apply_ufunc` (u_func refers to "user function").

### What is apply_ufunc?

`apply_ufunc` is a tool from the Xarray package that maps functions. It allows you to apply a function to every element of a DataArray while maintaining Xarray's attribute formatting and functionality. You can read more about `xarray.apply_ufunc` [here](http://xarray.pydata.org/en/stable/generated/xarray.apply_ufunc.html).

In this example the arguments to `apply_ufunc` are the function (`orig_units.convert`), then the input arguments for that function (the DataArray and target units) and `output_dtypes` (where we specify the datatype of the output as identical to the input datatype).

In [ ]:
orig_units = cf.Unit(ds['lev'].attrs['units'])
target_units = cf.Unit('km')

level_bounds_in_km = xr.apply_ufunc(orig_units.convert, ds.lev_bnds, target_units)
level_bounds_in_km

<h3 style="color:red"> Task 2 -  Write a function for unit conversion</h3>

This is only took us three lines of code to write, but you may want to check, convert, or assert desired units for every variable in a dataset. Your code will be much easier to read if this process is inside a function which you call up in one line. So let's write a function for unit conversion in the code cell block below:

In [ ]:
# Your code here

Here is my function:

In [ ]:
# %load solutions/solution_3_2.py

I used inputs of the dataset, the variable in that dataset, the variable bounds (because often the unit attribute is associated with the variable but you want to adjust values of the variable bounds too), and the target units. You may have used different inputs.

<h3 style="color:red"> Task 3 -  Call up your unit conversion function</h3>

Now use your function to assert your level bounds (`lev_bnds`) and temperature (`thetao`) DataSet variables units to that of meters ('m') and Kelvins ('degK') in the code cell block below:

In [ ]:
# Your code here

For my function, this looks as follows:

In [ ]:
# %load solutions/solution_3_3.py

## Going Futher:
 - Reading about CF conventions: http://cfconventions.org/
 - Reading about cf_units the package: https://scitools.org.uk/cf-units/docs/latest/unit.html
 - apply_ufunc documentation: http://xarray.pydata.org/en/stable/generated/xarray.apply_ufunc.html

<div class="alert alert-block alert-success">
  <p>Previous: <a href="02_subselecting_and_indexing_data.ipynb">Subselecting and Indexing Data</a></p>
  <p>Next: <a href="04_calculation_and_plotting.ipynb">OHC Calculation and Plotting</a></p>
</div>